# Assignment 1 2AMM10 2023-2024

## Group: [Fill in your group name]
### Member 1: [Fill in your name]
### Member 2: [Fill in your name]
### Member 3: [Fill in your name]

In [2]:
import os
import pandas as pd
import torch 
from torch.utils.data import Dataset
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from torchvision import transforms
import kagglehub
import torchvision.transforms as transforms

# add additonal imports here
class FashionDataset(Dataset):
    def __init__(self, csv_file, img_dir,column_class="articleTypeId", transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file with labels.
            img_dir (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = pd.read_csv(csv_file)  # load CSV file
        self.img_dir = img_dir  # image folder path
        self.transform = transform  # image transformations
        self.targets = list(self.df[column_class].values)


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.df.loc[idx,'imageId']}.jpg")  # Get image filename
        image = Image.open(img_name).convert("RGB")  # Load image

        if self.transform:
            image = self.transform(image)  # Apply transformations

        return image, self.targets[idx]

Download data with kagglehub

In [3]:
# import shutil
# import os

dataset_path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-small")
img_dir = os.path.join(dataset_path,"images")

# shutil.rmtree(dataset_path)

The different datasets can be loaded using the class `FashionDataset` which is a custon PyTorch dataset (see [Datasets & DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) for more information). Below an example of how to use the `FashionDataset` constructor as well as some visualizations. Please note that you may have to adapt the arguments to match the strucucture of your working directory.

In [4]:
dataset = FashionDataset("dataset/train.csv",img_dir)

In [5]:
dataset.df.head()

,imageId,articleTypeName,categoryName,articleTypeId,categoryId
0,35180,Backpacks,Bags,15,2
1,33585,Tshirts,Topwear,0,0
2,9464,Shirts,Topwear,1,0
3,8194,Shirts,Topwear,1,0
4,42231,Tops,Topwear,6,0


In [ ]:
np.random.seed(42)
random_indices = np.random.choice(len(dataset),3)

for i in random_indices:
    img, label = dataset[i]
    plt.title(dataset.df.iloc[i]["articleTypeName"]+f" ({label.item()})")
    plt.imshow(img)
    plt.show()

## Loading different datasets

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# transform = transforms.Compose([
#     transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(degrees=15),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

train_dataset = FashionDataset("dataset/train.csv",img_dir,transform=transform)
main_test_dataset =  FashionDataset("dataset/main_test.csv",img_dir,transform=transform)
new_test_dataset =  FashionDataset("dataset/new_test.csv",img_dir,transform=transform)
main_support_dataset =  FashionDataset("dataset/main_support.csv",img_dir,transform=transform)
new_support_dataset =  FashionDataset("dataset/new_support.csv",img_dir,transform=transform)
merged_test_dataset =  FashionDataset("dataset/merged_test.csv",img_dir,transform=transform) # merged corresponds to main+new
merged_support_dataset =  FashionDataset("dataset/merged_support.csv",img_dir,transform=transform)

# datasets with categories
main_test_dataset_cat =  FashionDataset("dataset/main_test.csv",img_dir,column_class="categoryId",transform=transform)
main_support_dataset_cat =  FashionDataset("dataset/main_support.csv",img_dir,column_class="categoryId",transform=transform)


label_id_to_label_name = {i: train_dataset.df[train_dataset.df["articleTypeId"]==i]["articleTypeName"].iloc[0] for i in range(39)} 
label_id_to_label_name.update({i: new_test_dataset.df[new_test_dataset.df["articleTypeId"]==i]["articleTypeName"].iloc[0] for i in range(39,39+30)})

## Task 1

In [ ]:
# your solution
# train dataset: train_dataset
# test datset: main_test_dataset

In [ ]:
# define network archietecture

class FashionNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(FashionNet, self).__init__()

        # convolutional layers
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size = 3, padding = 1)

        # batch normalization layers
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.bn2 = torch.nn.BatchNorm2d(64)
        self.bn3 = torch.nn.BatchNorm2d(128)

        # pooling layer
        self.pool = torch.nn.MaxPool2d(2, 2)

        # dropout layer
        self.dropout = torch.nn.Dropout(0.25)

        # fully conneted layer
        self.fc1 = torch.nn.Linear(128 * 28 * 28, 512)
        # self.fc1 = torch.nn.Linear(256 * 14 * 14, 512)
        self.fc2 = torch.nn.Linear(512, num_classes)

        # activation function
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        # First convolutional block
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Third convolutional block
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers with dropout
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x # already without softmax => can be used as a feature



Training

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.amp import autocast, GradScaler

# hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 1e-4
weight_decay = 1e-4

# create data loader
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(main_test_dataset, batch_size = batch_size, shuffle = False)

num_classes = len(train_dataset.df['articleTypeId'].unique())

# initialize the model
model = FashionNet(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Improved training function with AMP
def train_model(model, train_loader, test_loader, device, epochs=30):
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

    # Initialize gradient scaler for AMP
    scaler = GradScaler()
    
    # Track best model
    best_test_acc = 0.0
    
    print(f"Starting training for {epochs} epochs...")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            # Use AMP autocast
            with autocast(device_type='cuda', enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Scale gradients and perform backward pass
            scaler.scale(loss).backward()
            
            # Update weights with scaled gradients
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{(train_correct/train_total):.4f}'
            })
        
        train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        
        # Test phase
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        test_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{epochs} [Test]')
        
        with torch.no_grad():
            for inputs, labels in test_pbar:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
                
                test_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{(test_correct/test_total):.4f}'
                })
        
        test_loss = test_loss / len(test_loader)
        test_acc = test_correct / test_total
        
        # Update learning rate based on test accuracy
        scheduler.step(test_acc)
        
        # Print results
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

        # Clean GPU cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Save best model
        if test_acc > best_test_acc: 
            best_test_acc = test_acc
            # Save the model if needed
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"New best test accuracy: {test_acc:.4f}")

    print(f"Training completed! Best test accuracy: {best_test_acc:.4f}")
    return model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
print(f"Using device: {device}")
model = train_model(model, train_loader, test_loader, device, 10)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # No need to track gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Regular accuracy
    accuracy = 100 * correct / total
    # Balanced accuracy
    balanced_acc = 100 * balanced_accuracy_score(all_labels, all_preds)
    
    return accuracy, balanced_acc

# After training your model
acc, bal_acc = evaluate_model(model, test_loader, device)
print(f'Accuracy: {acc:.2f}%')
print(f'Balanced Accuracy: {bal_acc:.2f}%')

## Task 2

In [ ]:
# your solution
# train dataset: train_dataset
# test and support dataset: see scenarios table

In [ ]:
# triplet dataset

In [ ]:
from torch.utils.data import Dataset
import random

class TripletDataset(Dataset):
    def __init__(self, image_dataset):  # image_dataset = torchvision.datasets.ImageFolder or custom
        self.image_dataset = image_dataset
        self.labels = image_dataset.targets
        self.class_to_indices = self._map_labels_to_indices()

    def _map_labels_to_indices(self):
        from collections import defaultdict
        mapping = defaultdict(list)
        for idx, label in enumerate(self.labels):
            mapping[label].append(idx)
        return mapping

    def __getitem__(self, idx):
        anchor_img, anchor_label = self.image_dataset[idx]

        # Positive
        pos_idx = random.choice(self.class_to_indices[anchor_label])
        pos_img, _ = self.image_dataset[pos_idx]

        # Negative
        neg_label = random.choice([l for l in self.class_to_indices if l != anchor_label])
        neg_idx = random.choice(self.class_to_indices[neg_label])
        neg_img, _ = self.image_dataset[neg_idx]

        return anchor_img, pos_img, neg_img

    def __len__(self):
        return len(self.image_dataset)


In [ ]:
# new training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.amp import autocast, GradScaler

def train_embedding_model(model, triplet_loader, device, epochs=20, margin=1.0):
    model = model.to(device)
    criterion = nn.TripletMarginLoss(margin=margin)
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scaler = GradScaler()

    print(f"Starting embedding training for {epochs} epochs...")

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        pbar = tqdm(triplet_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for anchor, positive, negative in pbar:
            anchor, positive, negative = (
                anchor.to(device),
                positive.to(device),
                negative.to(device),
            )

            optimizer.zero_grad()

            with autocast(device_type='cuda', enabled=torch.cuda.is_available()):
                anchor_feat = model(anchor)
                positive_feat = model(positive)
                negative_feat = model(negative)

                loss = criterion(anchor_feat, positive_feat, negative_feat)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            pbar.set_postfix({"triplet_loss": f"{loss.item():.4f}"})

        avg_loss = running_loss / len(triplet_loader)
        print(f"Epoch {epoch+1}: Avg Triplet Loss = {avg_loss:.4f}")

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    print("Training complete.")
    return model

In [ ]:
# train model

In [ ]:
from torch.utils.data import DataLoader

triplet_dataset = TripletDataset(train_dataset)
triplet_loader = DataLoader(triplet_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)

In [ ]:
model = FashionNet(num_classes=39)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = train_embedding_model(model, triplet_loader, device, epochs=10)

In [ ]:
# save model

In [ ]:
torch.save(model.state_dict(), "trained_triplet_model.pth")

In [ ]:
# extract feature

In [ ]:
def extract_features(model, dataloader, device):
    model.eval()
    features, labels = [], []

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = model(inputs)  # 39D features
            features.append(outputs.cpu())
            labels.extend(targets.cpu().tolist())

    return torch.cat(features), labels

In [ ]:
# neighbor classify

In [ ]:
import torch.nn.functional as F

def classify_by_nearest_neighbor(test_features, support_features, support_labels, metric='cosine'):
    predictions = []

    for test_feat in test_features:
        if metric == 'cosine':
            sims = F.cosine_similarity(test_feat.unsqueeze(0), support_features)
            idx = torch.argmax(sims).item()
        if metric == 'euclidean':
            dists = torch.norm(support_features - test_feat.unsqueeze(0), dim=1)
            idx = torch.argmin(dists).item()

        predictions.append(support_labels[idx])
    
    return predictions

In [ ]:
# Scenario 1

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
main_test_loader = DataLoader(main_test_dataset, batch_size=32, shuffle=False)

support_features, support_labels = extract_features(model, train_loader, device)
test_features, test_labels = extract_features(model, main_test_loader, device)

preds = classify_by_nearest_neighbor(test_features, support_features, support_labels)
evaluate_predictions(test_labels, preds)

In [ ]:
# Scenario 2

In [ ]:
main_support_loader = DataLoader(main_support_dataset, batch_size=32, shuffle=False)

support_features, support_labels = extract_features(model, main_support_loader, device)
test_features, test_labels = extract_features(model, main_test_loader, device)

preds = classify_by_nearest_neighbor(test_features, support_features, support_labels)
evaluate_predictions(test_labels, preds)

In [ ]:
# Scenario 3

In [ ]:
new_support_loader = DataLoader(new_support_dataset, batch_size=32, shuffle=False)
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

support_features, support_labels = extract_features(model, new_support_loader, device)
test_features, test_labels = extract_features(model, new_test_loader, device)

preds = classify_by_nearest_neighbor(test_features, support_features, support_labels)
evaluate_predictions(test_labels, preds)

In [ ]:
# Scenario 4

In [ ]:
merged_support_loader = DataLoader(merged_support_dataset, batch_size=32, shuffle=False)
merged_test_loader = DataLoader(merged_test_dataset, batch_size=32, shuffle=False)

support_features, support_labels = extract_features(model, merged_support_loader, device)
test_features, test_labels = extract_features(model, merged_test_loader, device)

preds = classify_by_nearest_neighbor(test_features, support_features, support_labels)
evaluate_predictions(test_labels, preds)

## Task 3

In [ ]:
# your solution
# test dataset: merged_test_dataset
# support/catalog dataset: support_test_dataset

In [ ]:
# install what is needed

In [ ]:
!pip install matplotlib scikit-learn tqdm

In [ ]:
# import what is needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# simulate & label

In [ ]:
np.random.seed(42)

def simulate_embeddings(num_samples, dim=128):
    return np.random.randn(num_samples, dim)

def simulate_labels(num_samples, num_classes=10):
    return np.random.randint(0, num_classes, size=num_samples)

support_embeddings = simulate_embeddings(500)
support_labels = simulate_labels(500)

test_embeddings = simulate_embeddings(100)
test_labels = simulate_labels(100)

In [ ]:
# methods to compute top-k and confidence values

In [ ]:
def get_top_k_similarities(test_embedding, support_embeddings, k=3):
    similarities = cosine_similarity([test_embedding], support_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    return top_k_indices, similarities[top_k_indices]

def compute_confidence(similarities):
    return similarities[0] - similarities[2]

In [ ]:
# evaluate threshold

In [ ]:
thresholds = np.linspace(0, 1, 50)
error_rates = []
coverage_rates = []

for threshold in tqdm(thresholds):
    total = len(test_embeddings)
    shown = 0
    incorrect = 0

    for i, test_embedding in enumerate(test_embeddings):
        top_k_indices, similarities = get_top_k_similarities(test_embedding, support_embeddings)
        confidence = compute_confidence(similarities)

        if confidence >= threshold:
            shown += 1
            recommended_labels = support_labels[top_k_indices]
            if test_labels[i] not in recommended_labels:
                incorrect += 1

    coverage = shown / total if total else 0
    error_rate = incorrect / shown if shown else 0
    coverage_rates.append(coverage * 100)
    error_rates.append(error_rate * 100)

In [ ]:
# plot

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(error_rates, coverage_rates, marker='o', color='orange')
plt.xlabel('Error Rate (%)')
plt.ylabel('Coverage (%)')
plt.title('Error Rate vs. Coverage')
plt.grid(True)
plt.tight_layout()
plt.show()

## Task 4

In [ ]:
# your solution
# datasets: first 10 classes of train_dataset and main_test_dataset